<a href="https://colab.research.google.com/github/szrobi96/image_recognition/blob/main/Train_YOLO_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train YOLO Models in Google Colab
**Original Author:** Evan Juras, [EJ Technology Consultants](https://ejtech.io)

**Adapted by:** Róbert Szabó, [email](mailto:szabo.robert96@hotmail.com)

**Last updated:** July 08, 2025

**Original GitHub:** [Train and Deploy YOLO Models (Candies)](https://github.com/EdjeElectronics/Train-and-Deploy-YOLO-Models)

**Adapted GitHub:** [Train and Deploy YOLO Models (Pill or Capsule)](https://github.com/szrobi96/image_recognition.git)

# Introduction

This notebook uses [Ultralytics](https://docs.ultralytics.com/) to train YOLO11, YOLOv8, or YOLOv5 object detection models with a custom dataset. At the end of this Colab, you'll have a custom YOLO model that you can run on your PC, phone, or edge device like the Raspberry Pi.

Original YouTube video that walks through this guide step by step.

<p align=center>
<a href="https://youtu.be/r0RspiLG260" target="_blank"><img src="https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/doc/Train_YOLO_Thumbnail2.png" height="240"><br>
<i>Click here to go to the video!</i></a>
</p>

### Working in Colab
Colab provides a virtual machine in your browser complete with a Linux OS, filesystem, Python environment, and best of all, a free GPU. We'll install PyTorch and Ultralytics in this environment and use it to train our model. Simply click the Play button on sections of code in this notebook to execute them on the virtual machine.

### Navigation
To navigate this notebook, use the table of contents in the left sidebar to jump from section to section.

### Dataset
This notebook uses a custom dataset of images of pills and capsules. The dataset is available on Roboflow https://universe.roboflow.com/seblful/pills-detection-s9ywn and can be directly downloaded in the desired format. The dataset is split into training and validation sets, with proper annotations.

# Local Setup

The easiest way to run python code on a PC is using Anaconda (or Miniconda). Anaconda sets up a virtual Python environment and allows you to easily install Python and all necessery tools.

**1. Download and Install Anaconda**

Go to the Anaconda download page at https://www.anaconda.com/download/success#:~:text=Miniconda%20Installers, and then download the package for your OS. When it's finished downloading, run the installer and click through the installation steps. You can use the default options for installation.

**2. Set up virtual environment**

Once it's installed, run Anaconda Prompt from the Start Bar. (If you're on macOS or Linux, just open a command terminal).

Issue the following commands to create a new Python environment and activate it:

```shell
conda create --name yolo-env1 python=3.12 -y
conda activate yolo-env1
```

Install Ultralytics (which also installs import libraries like OpenCV-Python, Numpy, and PyTorch) by issuing the following command:

```shell
pip install ultralytics
```

**ONLY** If you have an NVIDIA GPU, you can install the GPU-enabled version of PyTorch by issuing the following command:

```shell
pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
```

**3. Download and run yolo_detect.py**

Create a new folder called `my_model` in your home directory (or wherever you want to store your model files). You can do this by running:

```shell
mkdir my_model
```
Then, change into the `my_model` directory:

```shell
cd my_model
```

Download the `yolo_detect.py` script into the `my_model` folder using:

```shell
curl -o yolo_detect.py https://raw.githubusercontent.com/szrobi96/image_recognition/refs/heads/main/yolo_detect.py
```

Ultralytics provides already trained models that you can use for object detection. For example, you can use `yolo11n.pt`, which is a small, fast model that can detect 80 different classes of objects on a Webcam by issuing the following command:
```python
python yolo_detect.py --model yolo11n.pt --source usb0 --resolution 1280x720
```

A window will appear showing a live feed from your webcam with boxes drawn around detected objects in each frame.

An overview of stock models can be found in the [Ultralytics Models documentation](https://docs.ultralytics.com/models/).

You can also run the model on an video file, image, or folder of images. To see a full list of arguments for `yolo_detect.py`, issue `python yolo_detect.py --help` or see the [README file](https://github.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/blob/main/README.md).

# Train YOLO Models in Google Colab with Custom Dataset

**Verify NVIDIA GPU Availability**

Make sure you're using a GPU-equipped machine by going to "Runtime" -> "Change runtime type" in the top menu bar, and then selecting one of the GPU options in the Hardware accelerator section. Click Play on the following code block to verify that the NVIDIA GPU is present and ready for training.

In [ ]:
!nvidia-smi

# 1.&nbsp;Gather and Label Training Images

Before we start training, we need to gather and label images that will be used for training the object detection model. A good starting point for a proof-of-concept model is 200 images. The training images should have random objects in the image along with the desired objects, and should have a variety of backgrounds and lighting conditions.

There are a couple options for gathering images:


*   Build a custom dataset by taking your own pictures of the objects and labeling them (this typically results in the best performance)
*   Find a pre-made dataset from sources like [Roboflow Universe](), [Kaggle](), or [Google Images V7]() <-- ***This is our preferred option for this notebook, as it allows you to quickly get started with a pre-made dataset.***

If you want to build your own dataset, there are several tools available for labeling images. One good option is [Label Studio](https://labelstud.io/?utm_source=youtube&utm_medium=video&utm_campaign=edjeelectronics), a free and open-source labeling tool that has a simple workflow while providing capabilities for more advanced features. My YouTube video that walks through this notebook (link to be added soon) shows how to label images with Label Studio.

<p align=center>
<img src="https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/doc/label-studio-example.PNG" height="380"><br>
<i>Example of a candy image labeled with Label Studio.</i>
</p>

* If you used Label Studio to label and export the images, they'll be exported in a `project.zip` file that contains the following:

- An `images` folder containing the images
- A `labels` folder containing the labels in YOLO annotation format
- A `classes.txt` labelmap file that contains all the classes
- A `notes.json` file that contains info specific to Label Studio (this file can be ignored)

## Download the Pill & Capsule Dataset from Roboflow

To train your model, you'll need a dataset of labeled images. The pill and capsule dataset is available for free on Roboflow Universe. Follow these steps to download it:

1. Go to the [Pills Detection Dataset on Roboflow Universe](https://universe.roboflow.com/seblful/pills-detection-s9ywn).
2. Click the **Dataset** button in the side panel.
3. Click **Download Dataset** and then again **Download Dataset** in the popup. Click **Continue**.
4. Under **Image and Annotation Format**, select your desired format (for this notebook, choose **YOLOv11**), check **Download zip to computer**, and click **Continue**.

A `.zip` file containing your dataset will be downloaded to your computer, please rename it manually to **data.zip**.

> **Tip:** If you have trouble downloading with Mozilla Firefox, try using Google Chrome or Microsoft Edge instead.

Once your dataset is downloaded, you're ready to move on to the next step!

# 2.&nbsp;Upload Image Dataset and Prepare Training Data

Next, we'll upload our dataset and prepare it for training with YOLO.

## 2.1 Upload images

First, we need to upload the dataset to Colab. Here are a few options for moving the `.zip` folder into this Colab instance.

**Upload through Google Colab**

Upload the `*.zip` file to the Google Colab instance by clicking the "Files" icon on the left hand side of the browser, and then the "Upload to session storage" icon. Select the zip folder to upload it.

<p>
<br>
<img src="https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/doc/upload-colab-files.png" height="240">
</p>


## 2.2 Split images into train and validation folders

At this point, whether you used Option 1, or 2, you should be able to click the folder icon on the left and see your `data.zip` file in the list of files. Next, we'll unzip `data.zip` and create some folders to hold the images. Run the following code block to unzip the data.

In [ ]:
# Unzip images to a custom data folder
!unzip -q /content/data.zip -d /content/data

Ultralytics requires a particular folder structure to store training data for models. The root folder is named “data”. Inside, there are two main folders:

*   **Train**: These are the actual images used to train the model. In one epoch of training, every image in the train set is passed into the neural network. The training algorithm adjusts the network weights to fit the data in the images.

*   **Validation**: These images are used to check the model's performance at the end of each training epoch.

... and a third, optional folder:

*   **Test**: These images are used to test the model's performance after training is complete. This set is not used during training, but is used to evaluate the model's accuracy on unseen data.

In each of these folders is a “images” folder and a “labels” folder, which hold the image files and annotation files respectively. The annotation files are in the YOLO format, which is a text file with the same name as the image file, but with a `.txt` extension. Each line in the text file contains the class ID and bounding box coordinates for an object in the image.

The data is already in the correct format, and split into train and validation sets, so we can simply continue with the next step.

# 3.&nbsp;Install Requirements (Ultralytics)

Next, we'll install the Ultralytics library in this Google Colab instance. This Python library will be used to train the YOLO model.

In [ ]:
!pip install ultralytics

# 4.&nbsp;Configure Training


There's one last step before we can run training: we need to create the Ultralytics training configuration YAML file. This file specifies the location of your train and validation data, and it also defines the model's classes. Since we downloaded the dataset from Roboflow, it should already be present. If you assembled the dataset another way, you may have to manually create the `data.yaml` configuration file.

In [ ]:
!cat /content/data/data.yaml

Adjust any information (for example the Path), if it not correct

# 5.&nbsp;Train Model

## 5.1 Training Parameters
Now that the data is organized and the config file is created, we're ready to start training! First, there are a few important parameters to decide on. Visit my article on [Training YOLO Models Locally](https://www.ejtech.io/learn/train-yolo-models) to learn more about these parameters and how to choose them.

**Model architecture & size (`model`):**

There are several YOLO11 models sizes available to train, including `yolo11n.pt`, `yolo11s.pt`, `yolo11m.pt`, `yolo11l.pt`, and `yolo11xl.pt`. Larger models run slower but have higher accuracy, while smaller models run faster but have lower accuracy. I made a brief YouTube video that compares performance of different YOLO models on a Raspberry Pi 5 and a laptop with a RTX 4050 GPU, [check it out here to get a sense of their speed accuracy](https://youtu.be/_WKS4E9SmkA). If you aren't sure which model size to use, `yolo11s.pt` is a good starting point.

You can also train YOLOv8 or YOLOv5 models by substituting `yolo11` for `yolov8` or `yolov5`.


**Number of epochs (`epochs`)**

In machine learning, one “epoch” is one single pass through the full training dataset. Setting the number of epochs dictates how long the model will train for. The best amount of epochs to use depends on the size of the dataset and the model architecture. If your dataset has less than 200 images, a good starting point is 60 epochs. If your dataset has more than 200 images, a good starting point is 40 epochs.


**Resolution (`imgsz`)**

Resolution has a large impact on the speed and accuracy of the model: a lower resolution model will have higher speed but less accuracy. YOLO models are typically trained and inferenced at a 640x640 resolution. However, if you want your model to run faster or know you will be working with low-resolution images, try using a lower resolution like 480x480.


## 5.2 Run Training!

Run the following code block to begin training. If you want to use a different model, number of epochs, or resolution, change `model`, `epochs`, or `imgsz`.

In [ ]:
!yolo detect train data=/content/data/data.yaml model=yolo11n.pt epochs=20 imgsz=512

The training algorithm will parse the images in the training and validation directories and then start training the model. At the end of each training epoch, the program runs the model on the validation dataset and reports the resulting mAP, precision, and recall. As training continues, the mAP should generally increase with each epoch. Training will end once it goes through the number of epochs specified by `epochs`.

> **NOTE:** Make sure to allow training to run to completion, because an optimizer runs at the end of training that strips out unneeded layers from the model.

The best trained model weights will be saved in `content/runs/detect/train/weights/best.pt`. Additional information about training is saved in the `content/runs/detect/train` folder, including a `results.png` file that shows how loss, precision, recall, and mAP progressed over each epoch.

# 6.&nbsp;Test Model

The model has been trained; now it's time to test it! The commands below run the model on the images in the validation folder and then display the results for the first 10 images. This is a good way to confirm your model is working as expected. Click Play on the blocks below to see how your model performs.

In [ ]:
!yolo detect predict model=runs/detect/train/weights/best.pt source=data/test/images save=True

In [ ]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')


The model should draw a box around each object of interest in each image. If it isn't doing a good job of detecting objects, here are a few tips:

1. Double-check your dataset to make sure there are no labeling errors or conflicting examples.
2. Increase the number of epochs used for training.
3. Use a larger model size (e.g. `yolo11l.pt`).
4. Add more images to the training dataset. See my [dataset video](https://www.youtube.com/watch?v=v0ssiOY6cfg) for tips on how to capture good training images and improve accuracy.

You can also run the model on video files or other images images by uploading them to this notebook and using the above `!yolo detect predict` command, where `source` points to the location of the video file, image, or folder of images. The results will be saved in `runs/detect/predict`.

Drawing boxes on images is great, but it isn't very useful in itself. It's also not very helpful to just run this models inside a Colab notebook: it's easier if we can just run it on a local computer. Continue to the next section to see how to download your newly trained model and run it on a local device.

# 7.&nbsp;Deploy Model

Now that your custom model has been trained, it's ready to be downloaded and deployed in an application! YOLO models can run on a wide variety of hardware, including PCs, embedded systems, and phones. Ultralytics makes it easy to convert the YOLO models to various formats (`tflite`, `onnx`, etc.) and deploy them in a variety of environments.

This section shows how to download the model and provides links to instructions for deploying it on your PC and edge devices like the Raspberry Pi.

## 7.1 Download YOLO Model

First, zip and download the trained model by running the code blocks below.

The code creates a folder named `my_model`, moves the model weights into it, and renames them from `best.pt` to `my_model.pt`. It also adds the training results in case you want to reference them later. It then zips the folder as `my_model.zip`.

In [ ]:
# Create "my_model" folder to store model weights and train results
!mkdir /content/my_model
!cp /content/runs/detect/train/weights/best.pt /content/my_model/my_model.pt
!cp -r /content/runs/detect/train /content/my_model

# Zip into "my_model.zip"
%cd my_model
!zip /content/my_model.zip my_model.pt
!zip -r /content/my_model.zip train
%cd /content

In [ ]:
# This takes forever for some reason, you can also just download the model from the sidebar
from google.colab import files

files.download('/content/my_model.zip')

## 7.2 Deploy YOLO Model on Local Devices

Next, we'll take our downloaded model and run it on a local device. This section provides instructions showing how to deploy YOLO models on various devices.

I wrote a basic Python script, `yolo_detect.py`, that shows how to load a model, run inference on an image source, parse the inference results, and display boxes around each detected class in the image. The [script](https://github.com/szrobi96/image_recognition/blob/main/yolo_detect.py) gives an example of how to work with Ultralytics YOLO models in Python, and it can be used as a starting point for more advanced applications.

### Deploy on PC (Windows, Linux, or macOS)

We can run the model on a PC using the `yolo_detect.py` script just like we did in the first section of this notebook. Follow these steps to run the model on your PC:

If you closed the Anaconda Prompt terminal, open it again and activate the `yolo-env1` environment by issuing:

```shell
conda activate yolo-env1
```

**1. Extract downloaded model**
Take the `my_model.zip` file you downloaded in Step 7.1 and unzip it to a folder on your PC where you already have the `yolo_detect.py` script. You can use any unzip tool, such as the built-in Windows or macOS unzip tool, or a third-party tool like 7-Zip.

**Optional Change directory to the model folder (if you closed the terminal before)**
In the Anaconda Prompt, change to the directory where you unzipped the model. You can do this by using the `cd` command followed by the path to the folder where you unzipped the model. For example, if you unzipped it to a folder called `my_model` in your home directory, you would use:

```shell
cd %USERPROFILE%\my_model
```

**2. Run the model on a webcam or video file**
Now that you're in the model folder, you can run the `yolo_detect.py` script to run inference on a webcam or video file. The script takes several arguments, including the model file, source (webcam or video file), and resolution.

To run the model on your webcam with the new model, use the following command:

```shell
python yolo_detect.py --model my_model.pt --source usb0 --resolution 1280x720
```
A window will appear showing a live feed from your webcam with boxes drawn around detected objects in each frame.

If you want to run the model on a video file, replace `usb0` with the path to your video file. For example download following video:

```shell
curl -o videos/test_video.mp4 https://raw.githubusercontent.com/szrobi96/image_recognition/main/objective/test_video.mp4
```

And then run:
```shell
python yolo_detect.py --model my_model.pt --source videos/test_video.mp4 --resolution 1280x720
```

You can also run the model on an video file, image, or folder of images. To see a full list of arguments for `yolo_detect.py`, issue `python yolo_detect.py --help` or see the [README file](https://github.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/blob/main/README.md).


# 8.&nbsp;Conclusion

Congratulations! You've successfully trained and deployed a YOLO object detection model. 😀

Next, you can extend your application beyond just drawing boxes and counting objects. Add functionality like logging the number of objects detected over time or taking a picture when certain objects are detected.

Thanks for working through this notebook, and good luck with your projects!